# Bloomberg BQuant Spotlight Webinar Series: Go with the Flow
This is a companion notebook to the "Deciphering Markets Using Fund Flows Data" webinar.

In [ ]:
import bql
import pandas as pd
import bqviz as bqv
import bqplot as bqp
import numpy as np

In [ ]:
bq = bql.Service()

In [ ]:
mkt_qry = """
LET(
    #FundFlow = REPLACENONNUMERIC(FUND_FLOW(DATES=RANGE(2015-01-01,2019-06-30),CURRENCY='USD')/10^9,0);
    
    #AssetClass = FUND_ASSET_CLASS_FOCUS;
    #Maturity = FUND_MATURITY_BAND_FOCUS;
    #GeoFocus = FUND_GEO_FOCUS;
    
    #Yr = YEAR(#FundFlow().DATE);
    #Mth = MONTH(#FundFlow().DATE);
    #Qtr = #Yr + ' ' + IF(#Mth<=3,'Q1',IF(#Mth<=6,'Q2',IF(#Mth<=9,'Q3','Q4')));
)

GET(
    SUM(
        GROUP(
              #FundFlow,
             [#AssetClass,#Maturity,#GeoFocus, #Yr,#Qtr,#Mth])
        ) as #Flows
) 
    
FOR(FILTER(FUNDSUNIV([ACTIVE,PRIMARY]),
            FUND_TYP==ETF)
    )
"""

In [ ]:
mkt_req = bq.execute(mkt_qry)

In [ ]:
mkt_df = mkt_req[0].df()
mkt_df.head()

In [ ]:
mkt_df[['#ASSETCLASS','#MATURITY','#GEOFOCUS']] = mkt_df[['#ASSETCLASS','#MATURITY','#GEOFOCUS']].fillna('Not Classified')

In [ ]:
asset_df = pd.pivot_table(mkt_df, index = '#QTR',columns='#ASSETCLASS',values='#Flows', aggfunc=np.sum)
asset_df.head()

In [ ]:
bqv.LinePlot(asset_df,legend='outside',title='Quarterly Asset Class Flows',y_label='Flows (Bil USD)').set_style().show()

In [ ]:
fi_df = pd.pivot_table(mkt_df[mkt_df['#ASSETCLASS']=='Fixed Income'], index = '#MATURITY',columns='#YR',values='#Flows', aggfunc=np.sum).fillna(0)
bqv.BarPlot(fi_df,padding=0.5,title='Maturity Focused Yearly Fixed Income Flows',y_label='Flows (Bil USD)').set_style().show()

In [ ]:
cum_eq_df = pd.pivot_table(mkt_df[mkt_df['#ASSETCLASS']=='Equity'], index = 'DATE',columns='#GEOFOCUS',values='#Flows', aggfunc=np.sum).fillna(0).cumsum(axis=0)
bqv.LinePlot(cum_eq_df,legend='outside',y_label='Flows (Bil USD)',title='Cumulative Monthly Equity Flows By Fund Focus').set_style().show()

## Search for funds that could help predict the weekly returns of the SPX

In [ ]:
spx_qry = """
LET(
    #SPXRtns = DROPNA(PCT_DIFF(PX_LAST(DATES=RANGE(2015-01-02,2019-06-28,FRQ=W),FILL=PREV)));
    
    #SPXPrc = PX_LAST(DATES=RANGE(2015-01-09,2019-06-28,frq=w),FILL=PREV);
)

GET(
    #SPXRtns,
    #SPXPrc
)

FOR('SPX Index'
)

"""

In [ ]:
spx_req = bq.execute(spx_qry)

In [ ]:
spx_df = pd.concat([spx_req[0].df().set_index('DATE',drop=True),spx_req[1].df().set_index('DATE',drop=True)['#SPXPrc']],axis=1)
spx_df = spx_df.reset_index()
spx_df.head(5)

In [ ]:
spy_qry = """
LET(
    #SPXRtns = VALUE(DROPNA(PCT_DIFF(PX_LAST(DATES=RANGE(2015-01-02,2019-06-28,FRQ=W),FILL=PREV))),['SPX Index']);
    
    #FlowDts= RANGE(2015-01-02,2019-06-21,FRQ=W);
    #Flows = FUND_FLOW(DATES=#FlowDts,PER=W,CURRENCY='USD');
    #Assets = FUND_TOTAL_ASSETS(DATES=#FlowDts,CURRENCY='USD',fill=prev);
    #FundPctChg = #Flows/#Assets;

)

GET(#SPXRtns,
    #FundPctChg,
    CORR(#FundPctChg,#SPXRtns)
)
    
FOR( 'SPY US EQUITY'
)

"""

In [ ]:
spy_req = bq.execute(spy_qry)
spy_df = spy_req[2].df()
spy_df.head()

In [ ]:
fund_srch_qry = """
LET(
    #SPXRtns = VALUE(DROPNA(PCT_DIFF(PX_LAST(DATES=RANGE(2015-01-02,2019-06-28,FRQ=W),FILL=PREV))),['SPX Index']);
    
    #FlowDts= RANGE(2015-01-02,2019-06-21,FRQ=W);
    #Flows = FUND_FLOW(DATES=#FlowDts,PER=W,CURRENCY='USD');
    #Assets = FUND_TOTAL_ASSETS(DATES=#FlowDts,CURRENCY='USD',fill=prev);
    #FundPctChg = #Flows/#Assets;
    
    #Corr = DROPNA(CORR(#FundPctChg,#SPXRtns));
    
    #Univ = FILTER(FUNDSUNIV([ACTIVE,PRIMARY]),
                    FUND_TYP==ETF AND FUND_INCEPT_DT < '2014-01-01' AND EXCH_CODE == 'US');
    
)

GET(
    NAME,
    FUND_TOTAL_ASSETS(CURRENCY='USD',FILL=PREV)/10^9 as #AUM,
    FUND_MGMT_STYLE,
    FUND_GEO_FOCUS,
    FUND_STRATEGY,
    FUND_ASSET_CLASS_FOCUS,
    FUND_INDUSTRY_FOCUS,
    FUND_MATURITY_BAND_FOCUS,
    FUND_LEVERAGE_TYPE,
    #Corr

)

FOR(FILTER(#Univ,
    GROUPRANK(#Corr,ORDER=DESC) <= 10
    )
)
"""

In [ ]:
fund_req = bq.execute(fund_srch_qry)

In [ ]:
fund_srch_dfs =[]

for resp in fund_req:
    df = resp.df()
    fund_srch_dfs.append(df)

cor_funds_df = pd.concat(fund_srch_dfs,axis=1)
cor_funds_df.sort_values('#Corr',ascending=False)

In [ ]:
cor_funds_df.index

In [ ]:
sel_fund_qry = """
LET(
    #FlowDts= RANGE(2015-01-02,2019-06-21,FRQ=W);
    #Flows = FUND_FLOW(DATES=#FlowDts,PER=W,CURRENCY='USD');
    #Assets = FUND_TOTAL_ASSETS(DATES=#FlowDts,CURRENCY='USD',fill=prev);
    #FundPctChg = #Flows/#Assets;
)

GET(
    #FundPctChg
)

FOR(['UJB US Equity', 'GLD US Equity', 'IYT US Equity', 'SCIF US Equity',
       'EDV US Equity', 'EWV US Equity', 'VEGI US Equity', 'TIP US Equity',
       'EMHY US Equity', 'RIGS US Equity']
)

"""

In [ ]:
sel_fund_req = bq.execute(sel_fund_qry)
sel_fund_df = sel_fund_req[0].df()
sel_fund_df = sel_fund_df.reset_index()
sel_fund_df = sel_fund_df.pivot(index = 'DATE',columns='ID',values='#FundPctChg').reset_index()
sel_fund_df['Avg Pct Flow Of AUM'] =sel_fund_df.mean(axis=1)
sel_fund_df = sel_fund_df.rename(columns={'DATE':'Fund Flow Date'})
sel_fund_df.head()

In [ ]:
fund_spx_df = pd.concat([sel_fund_df,spx_df],axis=1)
fund_spx_df.head()

In [ ]:
bqv.InteractiveScatterPlot(fund_spx_df).show()

In [ ]:
# Create some data to work with


colors=['#1B84ED', '#CF7DFF']

# Create scales
scale_x = bqp.DateScale()
scale_y1 = bqp.LinearScale()
scale_y2 = bqp.LinearScale()

# Create the Lines marks
mark_line1 = bqp.Lines(x=fund_spx_df['DATE'],
                       y=fund_spx_df['#SPXPrc'],
                       scales={'x': scale_x, 'y': scale_y1},
                       colors=[colors[0]],
                       labels=['SPX'],
                       display_legend=True)
mark_bar2 = bqp.Bars(x=fund_spx_df['DATE'],
                       y=fund_spx_df['Avg Pct Flow Of AUM'],
                       scales={'x': scale_x, 'y': scale_y2},
                       colors=[colors[1]],
                       labels=['Avg Flows'],
                       display_legend=True)

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Dates')
axis_y1 = bqp.Axis(scale=scale_y1,
                   orientation='vertical',
                   label='SPX Price',
                   side='right',
                   tick_style={'fill': colors[0]},
                   label_offset='3em')
axis_y2 = bqp.Axis(scale=scale_y2,
                   orientation='vertical',
                   label='Avg Pct Flow Of AUM',
                   grid_lines='none',
                   tick_style={'fill': colors[1]},
                   side='left',
                   label_offset='3em')

# Create Figure
figure = bqp.Figure(marks=[mark_bar2, mark_line1],
                    axes=[axis_x, axis_y1, axis_y2],
                    title='SPX vs 1 Week Previous Avg Pct Flow Of AUM of selected funds',
                    layout={'width':'100%', 'height': '400px'},
                    title_style={'font-size': '22px'},
                    legend_location='top-left',
                    legend_style={'stroke': 'none'},
                    fig_margin={'top': 50, 'bottom': 60,
                                'left': 90, 'right': 90})

# Display the figure
figure
